In [ ]:
from train import train_cross_val
import pandas as pd
import os

# suppress warnings due from sklearn metrics
# early in training, metrics such as F1 score are poorly defined
# as model does may not predict any positive classes
import warnings
warnings.filterwarnings("ignore")


# About This Notebook

* This notebook provides barebones functionality to reproduce the results for solubility classification models. (Table. X)
* It also provides the ability to tweak some hyperparameters for the training process.

In [ ]:
def run(config):
    out = []
    for i in range(0, 3):
        config['seed'] = i
        save_path = os.path.join("Trained_Models", f"{config['model']}", f'seed_{i}')
        val_metrics, test_metrics = train_cross_val(config, save_path=save_path)
        out.append(test_metrics)

    result_df = pd.concat(out, ignore_index=True)

    mean_results = result_df.mean()
    std_results = result_df.std()

    summary_df = pd.DataFrame({'mean': mean_results, 'std': std_results})
    summary_df = summary_df.sort_index()
    print(summary_df)

    result_df.to_csv(os.path.join("./results", config['model'] + '_results.csv'), index=False)

In [ ]:
# choose from the following models: resnet18, efficientnet, convnext

config = {
    "data_dir": "./Solubility-Data",  # path to the data directory
    "model": None,  # will be set in the loop
    "lr": 0.0005, 
    "batch_size": 128, 
    "weight_decay": 0.001,
    "center_crop": (1080, 1080), # center crop before resizing
    "resize": (224, 224), # resizing to 224x224
    "degrees": 0, # random rotation by 0 degrees 
    "translate": (0.1, 0.1), # random translation by 10%4
    "scale_lower": 0.95, # random scaling to a minimum 95% of original size
    "scale_upper": 1.4,  # random scaling to a maximum 140% of original size
    "num_epochs": 30, 
    'seed':0, 
    'device': 'cuda'
    }


for model in ['resnet18', 'efficientnet', 'convnext']:
    config['model'] = model
    run(config)